# Building a large scale coastal groundwater 3D model

In this tutorial we will build a large scale coastal groundwater 3D model using SEAWAT and its parallelized version imod-WQ. The goal of this exercise is to explain step by step the procedure in setting up the model itself as well as provide a bit of theoretical background into model development itself. Hopefully, this can serve as a blueprint for your future groundwater modelling endeavors!

### 1) Import Python libraries and define input/output directories

In this tutorial we will use the Flopy library to build the individual SEAWAT models. Apart from that, only standard Python libraries such as Numpy and Pandas are used.

In [1]:
#   who needs warnings.. we are now on day 3 of Python coding we know what we are doing 
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import libraries
import flopy
import numpy as np
import pandas as pd
import os
import pyvista as pv
import matplotlib
import xarray as xr
import PVGeo
import pyvistaqt
import imod
import flopy.utils.binaryfile as bf
import matplotlib
import matplotlib.pyplot as plt

C:\Users\zamrs001\AppData\Local\deltaforge\envs\3Dmodel_env\lib\site-packages\xugrid\ugrid\snapping.py:30: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos

Define the paths below based on where you stored the input files and the seawat executable.

In [3]:
# input and output directories
input_dir = r'g:\_Models\_NZ_Canterbury\_input'
out_dir = r'g:\_Models\_NZ_Canterbury\_model_run'
seawat_exe_dir = r'g:\_Models\_NZ_Canterbury\swt_v4x64.exe'

#   define the model name and model output directory
model_name = '_Canterbury_Model'
model_dir = os.path.join(out_dir, model_name)

# create the directories if they do not exist yet
os.makedirs(input_dir, exist_ok = True)
os.makedirs(out_dir, exist_ok = True)
os.makedirs(model_dir, exist_ok = True)

### 2) Define grid dimensions and create the initial grid

In the next steps we will create the 3D model grid. First we will define the grid cell dimensions (in meters) and then read in a CSV file that contains geological information and will be used to set up the 3D grid. This CSV file is created in Petrel as an ouput of a 3D geological model, but it could be any file with similar setup (e.g. Netcdf file). MODFLOW and SEAWAT basic model grid is called the IBOUND array - this is where we define active (1) and inactive (0) cells. In this tutorial we set all the cells based on their I J K indexes (column, row, layer) defined in the input CSV file. 

The input folder contains several CSV files, each for a different geological period - representing different depositional periods and stacking of lithological layers. The cmod_full.csv file represents the final geological environment (present time) and we will use it to set up our 3D IBOUND array. In such way we make sure that we have the full grid extent and can therefore accommodate all the geological layers. If we used the cmod_1.csv file that represents the geological environment during the fist geological period (i.e. the oldest geological layers) and its dimensions (at least in the vertical sense) might be smaller than the final geological environment. This could lead to a smaller IBOUND array where we could not fit the more recent geological layers. 

In [4]:
#   define grid dimensions in meters for row, col (dx and dy) and lay thickness (dz)
dx = dy = 1000
dz = 40

#   create the IBOUND array from the Petrel output, using the cmod_full, the fields list represents the column names of the CSV file
fields = ['I', 'J', 'K', 'X', 'Y', 'Z', 'Vsh']
#   use Pandas to read the CSV file and round the Z values (elevation) and the V_shale column
df_in = pd.read_csv(os.path.join(input_dir, 'cmod_full.csv'), header = 8, sep = " ", index_col = False, names = fields)
df_in['Z'] = df_in['Z'].round(1)
df_in['Vsh'] = df_in['Vsh'].round(2)

#   make a 3D numpy array with the dimensions from the input file
lay_arr = np.unique(df_in.K.values)  # unique layer numbers
row_arr = np.unique(df_in.I.values)  # unique row numbers
col_arr = np.unique(df_in.J.values)  # unique column numbers
#   numpy.zeros function create a 3D array where all cells are set to a value = 0 
full_ibound_arr = np.zeros((lay_arr.shape[0], row_arr.shape[0], col_arr.shape[0]))

#   get unique z values 
z_vals = np.unique(df_in.Z.values)
#   the max value will be the top elevation of the model
top_elev = np.nanmax(z_vals)  
#   have to do 2 * dz so it keeps the right end value
bot_elev = np.arange(top_elev, np.nanmin(z_vals) - 2 * dz, -dz)[1:]  

#   print the top, bottom elevations and the grid dimensions
print("Top elevation of the model grid      " + str(top_elev) + " m bsl.")
print("Bottom elevation of the model grid   " + str(bot_elev[-1]) + " m bsl.")
print("Total number of model layers         " + str(full_ibound_arr.shape[0]))
print("Total number of model rows           " + str(full_ibound_arr.shape[1]))
print("Total number of model columns        " + str(full_ibound_arr.shape[2]))

Top elevation of the model grid      -20.0 m bsl.
Bottom elevation of the model grid   -1420.0 m bsl.
Total number of model layers         35
Total number of model rows           154
Total number of model columns        160


### 3) Scenario definition 

Now that we have our main IBOUND grid dimensions and the array itself, we can think about what exactly we want our groundwater model to simulate. As we have multiple CSV files with geological environments at different times we can simulate two stress periods with different geologies and sea levels simulating a potential OFG deposition. Below we will define names of these stress periods, their length in years and corresponding sea levels.

In [5]:
#   define the model stress periods, names etc.
sp_names = ['cmod_3', 'cmod_4']  # just set it to match the csv names
sp_time_dur = [1000, 1000]  # duriation of each stress period - in years
sp_sea_level = [-130., 0.]  # sea level for each stress period

#   we start with the first stress period, Python is 0 based so 0 represents the first index..
a = 0

#   create the model directory for the stress period model
model_name_sp = sp_names[a]
sp_dir = os.path.join(model_dir, model_name_sp)
os.makedirs(sp_dir, exist_ok=True)
sea_level = sp_sea_level[a]  # get the sea level for the stress period

We will use the geology that was already exported in the CSV file - if you want to create a similar model to what is shown here you can use a similar procedure to build a groundwater model on top of your geological model. We also define a function that will reclassify the geological input based on the Vshale value into three lithological classes. You can of course increase the complexity if you wish but three different categories will be fine for our purposes here. Finally, we will assign a hydraulic conductivity value (in m/d) to all the classes. In this case we use a constant value for hydraulic conductivity which is a big simplification, in case you have a better geological model and hydraulic condcutivity estimation you can of course use that in the future. 

In [6]:
#   read in the csv file with active geology for the stress period
df_sp_in = pd.read_csv(os.path.join(input_dir, model_name_sp + '.csv'), header = 8, sep = " ", index_col = False, names = fields)

#   function that defines lithology based on porosity limit values
def get_lith_v2(data, res, non_res):
    """
    :param data: dataframe input
    :param res: lower porosity value treshold
    :param non_res: upper porosity value treshold
    """
    data['Code'] = pd.NaT
    data['Lith'] = pd.NaT
    #   sand is porosity higher than 0 but lower/equal than the res value
    data.loc[(data["Vsh"] <= res) & (data["Vsh"] > 0), ['Lith', 'Code']] = ['Sand', int(1)]
    #   shale has a high porosity equal or higher than non_res
    data.loc[data.Vsh >= non_res, ['Lith', 'Code']] = ['Shale', int(3)]
    #   mixed porosity is everything in between - so higher than res but lower than non_res
    data.loc[(data["Vsh"] > res) & (data["Vsh"] < non_res), ['Lith', 'Code']] = ['Mixed', int(2)]
    #   the rest will be assigned a 0 porosity
    data.loc[data.Code.isnull(), ['Lith', 'Code']] = ['NaN', int(0)]
    return data

#   add the lithology codes to the dataframe
df = get_lith_v2(df_sp_in, 0.3, 0.6)
df['hyd_k'] = pd.NaT
df['por'] = pd.NaT

#   insert hydraulic conductivity and porosity values into the dataframe, using theoretical values for the hydraulic conductivity
df.loc[df.Code == 1, ['hyd_k', 'por']] = [float(0.1), float(0.3)]
df.loc[df.Code == 2, ['hyd_k', 'por']] = [float(0.01), float(0.45)]
df.loc[df.Code == 3, ['hyd_k', 'por']] = [float(0.0001), float(0.60)]

### 4) Setting up the 3D model grid and geological conditions

Now we can create the IBOUND array, the Hydraulic conductivity (hk_arr) and the porosity arrasy (poro_arr) for the current stress periods. We will perform our first semi-illegal "for loop" in this tutorial and iterate through the rows of the CSV file to define active model cells and their hydraulic conductivity and porosity values - the rest of the IBOUND array (and the other arrays) will stay inactive.

In [11]:
#   now we can use this dataframe to fill in the ibound, poro and hk arrays as copy of the full ibound array
ibound_arr = np.copy(full_ibound_arr)
hk_arr = np.copy(full_ibound_arr)
poro_arr = np.copy(full_ibound_arr)

#   now gor row by row and if the hk_val is not nan or 0 then adapt the ibound (and other) array
df_sel = df.loc[df['Vsh'] > 0.]
for row in df_sel.iterrows():
    hk_arr[row[1]['K'] - 1, row[1]['I'] - 1, row[1]['J'] - 1] = row[1]['hyd_k']
    poro_arr[row[1]['K'] - 1, row[1]['I'] - 1, row[1]['J'] - 1] = row[1]['por']
    ibound_arr[row[1]['K'] - 1, row[1]['I'] - 1, row[1]['J'] - 1] = 1

Lets plot the 3D arrays and admire our effort so far! But first we need to create arrays with top and bottom elevation of each cell within the 3D grid. Since we work with a regular grid we can simply assign the same cell top and bottom elevation for each cell within a model layer. 

In [12]:
#   create a 3D array with the top and bottom cell elevation - we will use it for plotting the 3D array in the right scale (meters)
#   first create 1D list with cell top and bottom elevation (same for each layer since we have a regular grid)
top_arr_1d = np.arange(top_elev, bot_elev[-1], -dz)
bot_arr_1d = np.arange(bot_elev[0], bot_elev[-1] - dz, -dz)

#   now make 2D and 3D arrays that we will use for plotting
arr_2d = np.ones((hk_arr.shape[1], hk_arr.shape[2]))
top_arr_3d = np.ones((hk_arr.shape[0], hk_arr.shape[1], hk_arr.shape[2]))
bot_arr_3d = np.ones((hk_arr.shape[0], hk_arr.shape[1], hk_arr.shape[2]))

#   loop through the layers and assign the top/bottom elevation
for lay in range(top_arr_1d.shape[0]):
    top_arr_3d[lay, :, :] = arr_2d * top_arr_1d[lay]
    bot_arr_3d[lay, :, :] = arr_2d * bot_arr_1d[lay]

print("Created top and bottom elevation arrays with shape (layer, row, column) of " + str(top_arr_3d.shape[0]) + ' '+ str(top_arr_3d.shape[1]) + ' ' + str(top_arr_3d.shape[2]))

Created top and bottom elevation arrays with shape (layer, row, column) of 35 154 160


Now we can finally plot the 3D array of the hydraulic conductivity! First we will mask out the cells with hydraulic conductivity value of 0 (those are the inactive cells). In the next step, we will define a DataArray using the xarray Python library - you can imagine this as a netcdf file that would store hydraulic conductivity values for a 3D grid with 3 dimensions (layer, x, y). To plot the voxelized 3d grid we use a imod Python function - feel free to have a look into the source code if you feel extra nerdy! 

In [13]:
#   mask out the arrays where Hk = 0 m/d - those are the inactive cells
hk_arr_plot = np.copy(hk_arr)
hk_arr_plot[hk_arr_plot == 0] = np.nan

#   create an xarray from the z_array - has to fit the IMOD standards
da = xr.DataArray(data = hk_arr_plot,
                  dims = ["layer", "y", "x"],
                  coords = {"top" : (["layer", "y", "x"], top_arr_3d),
                            "bottom" : (["layer", "y", "x"], bot_arr_3d),
                            "layer" : np.arange(1, hk_arr_plot.shape[0] + 1),
                            "y" : np.arange(1, 1 + hk_arr_plot.shape[1]) * dy,
                            "x" : np.arange(1, 1 + hk_arr_plot.shape[2]) * dx})

#   create the voxels and plot the 3D grid
z_grid = imod.visualize.grid_3d(da, vertical_exaggeration=50, exterior_only=False, exterior_depth=1, return_index=False)
z_grid.plot(show_grid = True, window_size=[1600, 800])

C:\Users\zamrs001\AppData\Local\deltaforge\envs\3Dmodel_env\lib\site-packages\ipywidgets\widgets\widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
C:\Users\zamrs001\AppData\Local\deltaforge\envs\3Dmodel_env\lib\site-packages\ipywidgets\widgets\widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
C:\Users\zamrs001\AppData\Local\deltaforge\envs\3Dmodel_env\lib\site-packages\ipywidgets\widgets\widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Widget(value="<iframe src='http://localhost:55667/index.html?ui=P_0x28db9e9c910_1&reconnect=auto' style='width…

### 5) Initial conditions and stress period definition

To build our model we will use the Flopy Python package which is the official USGS Python library to build MODFLOW and SEAWAT models. As a first step to build the model we will define the directory where model input files will be stored. We will first run one stress period that will be 9999 years long - which is the maximum stress period length allowed in imod-wq (please do not ask me why..). The last line in this cell creates a SEAWAT model object in Python - think of it as a person getting ready to leave the house in the morning. In the next steps we will help them to dress up by assigning clothes, watch or jewelery representing different MODFLOW/SEAWAT model properties.

For more information you can check the official Flopy website - https://flopy.readthedocs.io/en/3.3.2/code.html we will use the MODFLOW 2005 and SEAWAT versions (NOT MODFLOW 6!), because imod-wq is tied to the MODFLOW 2005 version and MODFLOW 6 doesn't have parallel groundwater salinity module yet..

In [12]:
#   define the time discretization for the model run - we will run the model for 10 000 years
time_start = 0
time_end = 9999
#   define the modelname and folder for the mini stress period
mini_modelname = 'SP_' + str(time_start) + '_to_' + str(time_end)
mini_sp_dir = os.path.join(sp_dir, mini_modelname)

#   create the SEAWAT model object and start creating individual packages
mswt = flopy.seawat.Seawat(mini_modelname, 'nam_swt', model_ws = mini_sp_dir, exe_name = seawat_exe_dir)

Now that we have our 3D grid defined and filled it in with a geological information we can proceed to define the initial hydraulic head and salinity concentrations of our model. Ideally, we would have an estimated groundwater salinity and heads from a previous simulation or through an interpolation based on local/regional measurements. However, it is often the case that such information is not available so we need to make assumptions for the initial conditions. In this case we will first start with the whole domain being totally salinized and having groundwater heads equal to 0. 

In [13]:
#   define starting concentrations and starting heads, we will start with all cells being saline (35 TDS mg/l) 
sconc_arr = ibound_arr * 35.
strt_arr = ibound_arr * 0

#### DIS package
This is where we define the discretization (i.e. DIS) of the model domain - the number of layers, rows and columns as well as temporal discretization. 

In [14]:
#   make the DIS package, define the input first
nlay = lay_arr.shape[0]
nrow = row_arr.shape[0]
ncol = col_arr.shape[0]
delr = [dx] * ncol
delc = [dy] * nrow
perlen = [365.25 * 9999]
nstp = [10]
nper = len(perlen)
dis = flopy.modflow.ModflowDis(mswt, nlay, nrow, ncol, nper = 1, delr = delr, delc = delc, top = top_elev,
                               botm = bot_elev, perlen = perlen, nstp = nstp)

#### BAS package
Specify the active model cells in the domain (the IBOUND array) and the initial heads in these active cells.

In [15]:
#   next create the BAS package
bas = flopy.modflow.ModflowBas(mswt, ibound = ibound_arr, strt = strt_arr)

#### LPF package 
The layer property flow is used to define the hydraulic conductivities (the horizonatl and vertical conductivity) arrays. Here we use a uniform 

In [53]:
#   in this setup we will use the BCF package
vk_arr = hk_arr * 0.1
lpf = flopy.modflow.ModflowLpf(mswt, laytyp = 0, hk = hk_arr, vka = vk_arr, ipakcb = 1)

#### Boundary conditions - GHB, RCH, DRN
Now it is time to define some simple initial boundary conditions

- for the cells that are at the edge of our model domain we will define a GHB (general head boundary) condition which enables flow through these cells depending on a difference between the head in the adjacent cell and the GHB head elevation specified for the boundary cell. So if we specify the GHB head to be e.g. 10m the groundwater head in this cell will be kept 10m and if the groundwater head in the adjacent cells is lower then there will be inflow from the boundary. If the adjacent cells have a higher head than the GHB head elevation then there will be outflow through the boundary cell.
  
- for the cells where the top elevation is above the specified sea level we will apply freshwater recharge (RCH) of a constant value. In case you want to specify different recharge periods you need to setup multiple stress periods.
  
- to ensure that there is a simple surface drainage system in place (since we do not simulate rivers or lakes in this case) we implement a simple drainage network (DRN) by defining a drain elevation which limits the groundwater head elevation in this cell. This ensures that we do not inject fresh water into the model domain via RCH. 

In this process we create the ICBUND array where we specify boundary cells (= -1), this an input into the MODFLOW/SEAWAT model. Each of these boundaries also need to have a specified concentration, therefore we create a SSM list where each boundary/recharge cell will have a specified concentration.

In [54]:
#   create the icbund array
icbund_arr = ibound_arr

In [57]:
#   create the GHB package input here, also start creating the SSM package
itype = flopy.mt3d.Mt3dSsm.itype_dict()
ghb_input_lst = []
chb_input_lst = []
ssmdata = []
cond_const = 1000000.
#   the inland part on the edges of the active model domain will be assigned the topographical head
#   for each row check the first active cell - and if it is above sea level then assign fresh head
col_idx = [0, ncol - 1]
for i in range(nrow):
    #   select the active cells only
    for col in col_idx:
        row_cells = [t for t in ibound_arr[:, i, col].tolist() if t == 1]
        if len(row_cells) > 0:
            lay_idx = ibound_arr[:, i, col].tolist().index(1)
            if ibound_arr[lay_idx, i, col] == 1 and bot_elev[lay_idx] + dz >= sea_level:
                #   check if the top elevation of the cell is above sea level (thats why we do bot_elev + dz)
                for k in range(nlay):
                    if ibound_arr[k, i, col] == 1:
                        cond_val = hk_arr[k, i, col] * dz * 1000
                        ghb_input_lst.append([k, i, col, bot_elev[k] + dz, cond_val])
                        ssmdata.append([k, i, 0, 0.0, itype['GHB']])
                        icbund_arr[k, i, col] = -1
#   now do the same but for columns
row_idx = [0, nrow - 1]
for i in range(ncol):
    #   select the active cells only
    for row in row_idx:
        col_cells = [t for t in ibound_arr[:, row, i].tolist() if t == 1]
        if len(col_cells) > 0:
            lay_idx = ibound_arr[:, row, i].tolist().index(1)
            if ibound_arr[lay_idx, row, i] == 1 and bot_elev[lay_idx] + dz >= sea_level:
                for k in range(nlay):
                    if bot_elev[k] + dz >= sea_level:
                        cond_val = hk_arr[k, row_idx, i] * dz * 1000
                        ghb_input_lst.append([k, row_idx, i, bot_elev[k] + dz, cond_val])
                        ssmdata.append([k, row_idx, i, 0.0, itype['GHB']])
                        icbund_arr[k, row_idx, i] = -1
#   now check for the offshore domain and set all the cells below sea level to saltwater concentration and
#   head equal to sea level. Only in the top layer
for i in range(nrow):
    for j in range(ncol):
        lay_cells = [t for t in ibound_arr[:, i, j].tolist() if t == 1]
        if len(lay_cells) > 0:
            lay_idx = ibound_arr[:, i, j].tolist().index(1)
            if ibound_arr[lay_idx, i, j] == 1 and bot_elev[lay_idx] + dz < sea_level:
                cond_val = (vk_arr[lay_idx, i, j] * dx * dy) / dz
                ghb_input_lst.append([lay_idx, i, j, sea_level, cond_val])
                ssmdata.append([lay_idx, i, j, 35.0, itype['GHB']])
#   write the final output dictionary, inlcude each stress period
ghb_arr_in = {}
for d in range(len(perlen)):
    ghb_arr_in[d] = ghb_input_lst
ghb = flopy.modflow.ModflowGhb(mswt, ipakcb = 1, stress_period_data = ghb_arr_in)

In [60]:
#   the RCH package
rch_val = 0.00025
rch_arr = np.zeros((1, ibound_arr.shape[1], ibound_arr.shape[2]))
#   only apply recharge to the cells above sea level, for each column find the first active layer
for i in range(ibound_arr.shape[1]):
    for j in range(ibound_arr.shape[2]):
        lay_lst = [t for t in ibound_arr[:, i, j].tolist() if t == 1]
        if len(lay_lst) > 0:
            top_act_lay = ibound_arr[:, i, j].tolist().index(1)
            top_act_elev = top_elev - top_act_lay * dz
            #   if the top elevation of the cell is above sea level then assign recharge to it
            if top_act_elev >= sea_level:
                #print(top_act_elev)
                rch_arr[0, i, j] = rch_val
rch = flopy.modflow.ModflowRch(mswt, nrchop = 3, ipakcb = 1, rech = rch_arr)

In [61]:
#   the DRN package is assigned only to cells that receive recharge - cells with elev above sea level
drn_input_lst = []
for i in range(ibound_arr.shape[1]):
    for j in range(ibound_arr.shape[2]):
        lay_lst = [t for t in ibound_arr[:, i, j].tolist() if t == 1]
        if len(lay_lst) > 0:
            top_act_lay = ibound_arr[:, i, j].tolist().index(1)
            top_act_elev = top_elev - top_act_lay * dz
            #   if the top elevation of the cell is above sea level then assign recharge to it
            if top_act_elev >= sea_level:
                cond_cell = (vk_arr[top_act_lay, i, j] * dx * dy) / dz
                drn_input_lst.append([int(top_act_lay), i, j, top_act_elev, cond_cell])
#   write the final output dictionary, inlcude each stress period
if len(drn_input_lst) > 0:
    drn_arr_in = {}
    for c in range(len(perlen)):
        drn_arr_in[c] = drn_input_lst
    drn = flopy.modflow.ModflowDrn(mswt, ipakcb=1, stress_period_data=drn_arr_in)

#### OC package
This step will help us define the frequency of model output to be stored (concentration, heads, flow vectors etc.). 

In [62]:
#   write the OC package
ihedfm = 1  # a code for the format in which heads will be printed.
iddnfm = 0  # a code for the format in which drawdowns will be printed.
extension = ['oc', 'hds', 'ddn', 'cbc']
unitnumber = [14, 30, 52, 51]
#   create the dictionary that defines how to write the output file
spd = {(0, 0): ['SAVE HEAD', 'SAVE BUDGET', 'PRINT HEAD', 'PRINT BUDGET', 'SAVE HEADTEC', 'SAVE CONCTEC',
                'SAVE VXTEC', 'SAVE VYTEC', 'SAVE VZTEC']}
for t in range(0, nper):
    per = t  # + 1
    #   to save space on disk, every 10th timestep is saved
    for g in range(1, nstp[t] + 1):
        spd[(per, int(g))] = ['SAVE HEAD', 'SAVE BUDGET', 'PRINT HEAD', 'PRINT BUDGET', 'SAVE HEADTEC', 'SAVE CONCTEC',
                              'SAVE VXTEC', 'SAVE VYTEC', 'SAVE VZTEC']
oc = flopy.modflow.ModflowOc(mswt, ihedfm=ihedfm, stress_period_data=spd, unitnumber=unitnumber, compact=True)

#### BTN package 
Package to define the initial concentration and porosity.. det0 = 0 means that SEAWAT/imod-wq will automatically compute the largest time step possible and therefore make sure the model is as fast as possible. 

In [63]:
#   the BTN package
porosity = poro_arr
dt0 = 365.25
nprs = 1
ifmtcn = 0
chkmas = False
nprmas = 10
nprobs = 10
timprs_lst = list(np.linspace(1., perlen[0], 10, endpoint=True, dtype=int))
btn = flopy.mt3d.Mt3dBtn(mswt, nprs=nprs, timprs=timprs_lst, prsity=porosity, sconc=sconc_arr,
                         ifmtcn=ifmtcn, chkmas=chkmas, nprobs=nprobs, nprmas=nprmas, dt0=0)

#### ADV package 
Here we define the solver we will use in the model - mixelm (int) – MIXELM is an integer flag for the advection solution option.

- MIXELM = 0, the standard finite-difference method with upstream or central-in-space weighting, depending on the value of NADVFD;
- = 1, the forward-tracking method of characteristics (MOC);
- = 2, the backward-tracking modified method of characteristics (MMOC);
- = 3, the hybrid method of characteristics (HMOC) with MOC or MMOC automatically and dynamically selected;
- = -1, the third-order

We will use the simplest finite-difference method since it is the fastest - but it is also the most simple and probably wrong solver to use in general..

In [64]:
#   write the ADV package
adv = flopy.mt3d.Mt3dAdv(mswt, mixelm=0, mxpart=2000000)

#### DSP package
This is where we define the dispersivity and diffusion coefficients..

In [65]:
#   write the DSP package
dmcoef = 0.0000864  # effective molecular diffusion coefficient [M2/D]
al = 1.
trpt = 0.1
trpv = 0.1
dsp = flopy.mt3d.Mt3dDsp(mswt, al=al, trpt=trpt, trpv=trpv, dmcoef=dmcoef)

#### VDF package

In [66]:
#   write the VDF package
iwtable = 0
densemin = 1000.
densemax = 1025.
denseref = 1000.
denseslp = 0.7143
firstdt = 0.001
vdf = flopy.seawat.SeawatVdf(mswt, iwtable=iwtable, densemin=densemin, densemax=densemax,
                             denseref=denseref, denseslp=denseslp, firstdt=firstdt)

#### SSSM package 
Here we define the boundary conditions concentration values.

In [67]:
#   write the SSM package
ssm_rch_in = np.copy(rch_arr) * 0.0
ssmdata_dict = {0: ssmdata, 1: ssmdata}
ssm = flopy.mt3d.Mt3dSsm(mswt, crch=ssm_rch_in, stress_period_data=ssmdata_dict)

#### Finally now we can use this one line to create all the model input files.
After you run the line go to your model folder and you should see modflow files being created.

In [68]:
#   write packages and run model
mswt.write_input()

Util2d rech_1: locat is None, but model does not support free format and how is internal... resetting how = external


#### Additionally, to use imod-wq, we also need to create files that will tell it what solvers to use and what grid to use to split the models into individual partitions.

In [69]:
#   write the ascii file with vertical sum of active cells in IBOUND
ibound_arr_sum = np.sum(ibound_arr, axis=0, dtype=np.int32)
ibound_arr_sum = ibound_arr_sum.astype(str)
with open(os.path.join(mini_sp_dir, 'LOAD.ASC'), 'wb') as f:
    f.write(ibound_arr_sum)

#   create the pksf and pkst files - change it in case the grid discretization changes
pksf_lines = ['ISOLVER 1', 'NPC 2', 'MXITER 200', 'RELAX .98', 'HCLOSEPKS 0.01', 'RCLOSEPKS 10000.0', 'PARTOPT 0',
              'PARTDATA', 'external 40 1. (free) -1', 'GNCOL 274', 'GNROW 223', 'GDELR', '1000', 'GDELC', '1000',
              'NOVLAPADV 2', 'END']
pkst_lines = ['ISOLVER 2', 'NPC 2', 'MXITER 1000', 'INNERIT 50', 'RELAX .98', 'RCLOSEPKS 1.0E-05',
              'HCLOSEPKS 1.0E+12', 'RELATIVE-L2NORM', 'END']
# 'CCLOSEPKS=0.00001'

with open(os.path.join(mini_sp_dir, mini_modelname + '.pksf'), 'w') as f:
    for line in pksf_lines:
        f.write(line)
        f.write('\n')

with open(os.path.join(mini_sp_dir, mini_modelname + '.pkst'), 'w') as f:
    for line in pkst_lines:
        f.write(line)
        f.write('\n')

#   open the nam_swt file and append these three lines
nam_lines = ['PKSF          	  27 ' + mini_modelname + '.pksf', 'PKST              35 ' +
             mini_modelname + '.pkst', 'DATA 40 LOAD.ASC']

with open(os.path.join(mini_sp_dir, mini_modelname + '.nam_swt'), 'a') as f:
    for line in nam_lines:
        f.write(line)
        f.write('\n')

#   save the ibound_arr (if it doesnt exist)
ibound_arr_dir = os.path.join(sp_dir, 'ibound_arr.npy')
if not os.path.isfile(ibound_arr_dir):
    np.save(ibound_arr_dir, ibound_arr)

### 5) Running the SEAWAT/imod-wq model in Windows

The SEAWAT executables - all is in _3Dmodel/imodwq folder.

Place all the files either into the model directory, or make a C:Drive folder and put them there (e.g. C:\SEAWAT). There are two batch files (run.bat and run_parallel.bat), these are for single-core or parallel runs respectively – therefore use accordingly. Note that the name file (ending with ‘.nam_swt’) in the batch file will need to be adjusted according to your model name.  Within the batch file, the location of the SEAWAT executables will also need to be updated if they are not in your model directory. 

Message Passing Interface (MPICH2) - MPICH is a tool used for distributed-memory applications used in parallel computing. It needs to be installed to run SEAWAT in parallel. The MPICH2 installation file is located in: _3Dmodel/imodwq folder.

After installation, make sure that the batch file (run_parallel.bat) file you are using is linked to the right installation location.cation.


When the model run is finished you will see the following files created in your folder. As you can see we used 8 cores so the model domain was split into 8 partitions. Now we need to put the puzzle back together so we can examine the groundwater salinity for the whole model domain.

In [70]:
#   create the results directory if it doesnt exist
main_res_dir = os.path.join(model_dir, 'results')
os.makedirs(main_res_dir, exist_ok = True)

#   create the main output folders
netcdf_dir = os.path.join(main_res_dir, '_netcdf', model_name_sp)
os.makedirs(netcdf_dir, exist_ok=True)

In [71]:
#   get the nlay, nrow and ncol values
nlay, nrow, ncol = ibound_arr.shape[0], ibound_arr.shape[1], ibound_arr.shape[2]

To put the puzzle back together we will loop through the individual partitions and glue them back together into the big 3D array of the whole model domain. Here we will open the "list" file (open it in your text editor) where the grid partitioning is specified.

In [17]:
#   Extract the salinity concentrations and heads from the partition list and UCN files
#   find the list.p0000 file
partition_idxs_lst = []
str_start = ' p000 :   '
str_end = ' p007 :   '
if os.path.isfile(os.path.join(mini_sp_dir, mini_modelname + '.list.p000')):
    with open(os.path.join(mini_sp_dir, mini_modelname + '.list.p000'), 'r') as listfile:
        lines = listfile.readlines()
        #   loop through a list of core numbers
        for i in range(24):
            if i < 10:
                str_i = '0' + str(i)
            else:
                str_i = str(i)
            #   look for the line
            for row in lines:
                if row.find(' p0' + str_i + ' :   ') == 0:
                    idx_lst = [int(i) for i in row.split('|')[1].split()]
                    #   append to the list of partition_idxs_lst
                    #   the format: partition, partition_str, row_start, row_end, col_start, col_end
                    partition_idxs_lst.append([i, 'p0' + str_i, idx_lst[-2], idx_lst[-1],
                                               idx_lst[-4], idx_lst[-3]])

#    print the file location to open in your text editor
print("Please open this file in your text editor!   " + os.path.join(mini_sp_dir, mini_modelname + '.list.p000'))

NameError: name 'mini_sp_dir' is not defined

Now we read the binary file with the model output, and print the time steps.

In [73]:
#   read the first UCN file to get time steps
ucnobj = bf.UcnFile(os.path.join(mini_sp_dir, 'MT3D001.UCN.p000'))
time_steps = ucnobj.get_times()
print("TIME STEPS ", time_steps)

TIME STEPS  [1.0, 405790.0, 811590.0, 1217400.0, 1623200.0, 2029000.0, 2434800.0, 2840500.0, 3246300.0, 3652100.0]


Next, create the output arrays for each partition for the given time step.

In [74]:
#   only create the output for the last time step
ts = time_steps[-1]

#   make a 3D array with dimension of nlay (n partitions) and nrow and ncol
partition_arr = np.zeros((len(partition_idxs_lst), nlay, nrow, ncol)) * np.nan
partition_head_arr = np.zeros((len(partition_idxs_lst), nlay, nrow, ncol)) * np.nan
#   now loop through the partition indxs list and assign the partition array to the corresponding vertical layer
for k in range(len(partition_idxs_lst)):
    #   read in the UCN file and get the right timestep
    ucnobj = bf.UcnFile(os.path.join(mini_sp_dir, 'MT3D001.UCN.' + partition_idxs_lst[k][1]))
    time_steps = ucnobj.get_times()
    conc_arr = ucnobj.get_data(totim=ts).astype(dtype=np.float64)
    conc_arr[conc_arr > 100.] = np.nan
    partition_arr[k, :, partition_idxs_lst[k][2] - 1 : partition_idxs_lst[k][3],
                  partition_idxs_lst[k][4] - 1 : partition_idxs_lst[k][5]] = conc_arr
    nrow_part, ncol_part = conc_arr.shape[1], conc_arr.shape[2]

    #   read the heads from the list file as well
    #with open(os.path.join(mini_sp_dir, mini_modelname + '.list.p000'), 'r') as listfile:
    with open(os.path.join(mini_sp_dir, mini_modelname + '.list.' + partition_idxs_lst[k][1]), 'r') as listfile:
        lines = listfile.readlines()
        #   create a head array for the partition and find the corresponding lines
        head_arr = np.zeros((nlay, nrow_part, ncol_part)) * np.nan
        #   loop through layers
        for z in range(nlay):
            #   now look for the string in the list file
            lay_ts_str = 'HEAD IN LAYER' + str(z + 1).rjust(4) + ' AT END OF TIME STEP' + str(time_steps.index(ts) + 1).rjust(4)
            #print(lay_ts_str)
            lay_str_end = '1'
            #   find the starting and end index
            for row in lines:
                if row.find(lay_ts_str) > 0:
                    st_idx = lines.index(row)
                    break
            for row in lines[st_idx:]:
                if row.find(lay_str_end) == 0:
                    end_idx = st_idx + lines[st_idx:].index(row)
                    break
            #   loop through the selected lines and get all the row, col values into the head_arr
            #   first, create a list of row indexes that we will then loop through to extract the data
            row_idx_lst = []
            lines_sel = lines[st_idx : end_idx]
            for y in range(nrow_part):
                for row in lines_sel:
                    if row.find(str(y + 1).rjust(4)) == 0:
                        row_idx_lst.append(lines_sel.index(row))
            #   since it is a regular grid the number of rows in the text file will always be the same
            #   so calculate the row step and then use it to get the actuall head values.. finally
            row_step = row_idx_lst[1] - row_idx_lst[0]
            for row in row_idx_lst:
                lines_row = lines_sel[row : row + row_step]
                heads_col_lst = []
                for line in lines_row:
                    if lines_sel.index(line) == row:
                        #   append the values but skip the row index which is always the first string
                        heads_col_lst.append([float(num) for num in line.split()][1:])
                    else:
                        heads_col_lst.append([float(num) for num in line.split()])
                final_head_lst = [item for sublist in heads_col_lst for item in sublist]
                #   insert the list into the head_arr
                head_arr[z, row_idx_lst.index(row), :] = final_head_lst
    #   add the partition head array into the overall head array
    partition_head_arr[k, :, partition_idxs_lst[k][2] - 1 : partition_idxs_lst[k][3],
                       partition_idxs_lst[k][4] - 1 : partition_idxs_lst[k][5]] = head_arr

There is always an overlay on the edges of each partition - in these overlapping cells we will take the mean concentration (and heads). So we average the arrays along an axis..

In [75]:
#   create a nanmean array from the partition_arr - along the axis = 0 (which is the number of partitions)
final_arr = np.nanmean(partition_arr, axis=0)
final_head_arr = np.nanmean(partition_head_arr, axis=0)

Create output netcdf and numpy dictionary files - these can be used in case you want to run a subsequent stress period. In such case you would use these files to load the initial heads and concentration for the next stress period.

In [76]:
#   create the output files
sconc_dir = os.path.join(mini_sp_dir, '_sconc_arr.npy')
strt_dir = os.path.join(mini_sp_dir, '_strt_arr.npy')
np.save(sconc_dir, final_arr, allow_pickle=True)
np.save(strt_dir, final_head_arr, allow_pickle=True)
#   make the netcdf files
tot_time_str = str(sp_time_dur[a] * a + time_end)
conc_nc_dir = os.path.join(netcdf_dir, '_conc_time_' + tot_time_str + '_yrs.nc')
head_nc_dir = os.path.join(netcdf_dir, '_head_time_' + tot_time_str + '_yrs.nc')
#   create the netcdf files
x_coord_lst = np.arange(dx / 2, final_arr.shape[2] * dx + dx / 2, dx).tolist()
y_coord_lst = np.arange(dy / 2, final_arr.shape[1] * dy + dy / 2, dy).tolist()
z_coord_lst = np.linspace(0, final_arr.shape[0], final_arr.shape[0]).tolist()
conc_nc = xr.Dataset(data_vars={'salinity': (('z', 'y', 'x'), final_arr)},
                     coords={'x': x_coord_lst,
                             'y': y_coord_lst,
                             'z': z_coord_lst,
                             'time': sp_time_dur[a] * a + time_end})
conc_nc.to_netcdf(conc_nc_dir)
head_nc = xr.Dataset(data_vars={'gw_head': (('z', 'y', 'x'), final_head_arr)},
                     coords={'x': x_coord_lst,
                             'y': y_coord_lst,
                             'z': z_coord_lst,
                             'time': sp_time_dur[a] * a + time_end})
head_nc.to_netcdf(head_nc_dir)

And finally, we will plot the concentration!

In [92]:
#   create an xarray from the z_array - has to fit the IMOD standards
da = xr.DataArray(data = final_arr,
                  dims = ["layer", "y", "x"],
                  coords = {"top" : (["layer", "y", "x"], top_arr_3d),
                            "bottom" : (["layer", "y", "x"], bot_arr_3d),
                            "layer" : np.arange(1, hk_arr_plot.shape[0] + 1),
                            "y" : np.arange(1, 1 + hk_arr_plot.shape[1]) * dy,
                            "x" : np.arange(1, 1 + hk_arr_plot.shape[2]) * dx})

#   define cmap
cmap = plt.cm.jet  # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# create the new map
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('Salinity cmap', cmaplist, cmap.N)
# define the bins and normalize
bounds = np.array([0., 0.1, 0.25, 0.5, 1, 2.5, 5, 10., 15., 25., 35.])
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)

#   create the voxels and plot the 3D grid
z_grid = imod.visualize.grid_3d(da, vertical_exaggeration=50, exterior_only=False, exterior_depth=1, return_index=False)
z_grid.plot(show_grid = True, window_size=[1600, 800], cmap = cmap)

Widget(value="<iframe src='http://localhost:52062/index.html?ui=P_0x27fa217f790_11&reconnect=auto' style='widt…